In [1]:
from azureml.core import Workspace

# set up workspace
ws = Workspace.from_config()

# Take a look at Workspace
ws.get_details()

# set up datastores
dstore = ws.get_default_datastore()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Subscription Id: ' + ws.subscription_id, 
      'Resource Group: ' + ws.resource_group, 
      sep = '\n')

Workspace Name: ManyModelsAccelerator
Azure Region: westus2
Subscription Id: f97fb87f-32d7-4d7c-9bc5-ea43b4fea7ac
Resource Group: ManyModelsRG


In [2]:
from azureml.core import Experiment

experiment = Experiment(ws, 'github_training_pipeline')

print('Experiment name: ' + experiment.name)

Experiment name: github_training_pipeline


In [9]:
from azureml.core.dataset import Dataset

dataset_name = 'oj_data_small'
small_dataset = Dataset.get_by_name(ws, name=dataset_name)
small_dataset_input = small_dataset.as_named_input('oj_dataset')

#dataset = Dataset.get_by_name(ws, name='oj_data')
#dataset_input = dataset.as_named_input('train_all_models')

In [10]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment(name="many_models_environment")
train_conda_deps = CondaDependencies.create(pip_packages=['sklearn', 'pmdarima'])
train_env.python.conda_dependencies = train_conda_deps

In [12]:
from azureml.contrib.pipeline.steps import ParallelRunConfig
from azureml.core.compute import AmlCompute

compute = AmlCompute(ws, "hichandonew")
process_count_per_node = 8
node_count = 5
timeout = 500

tags = {}
tags['dataset_name'] = dataset_name
tags['node_count'] = node_count
tags['process_count_per_node'] = process_count_per_node
tags['timeout'] = timeout

parallel_run_config = ParallelRunConfig(
    source_directory='./scripts',
    entry_script='train.py',
    mini_batch_size="1",
    run_invocation_timeout=timeout,
    error_threshold=10,
    output_action="append_row",
    environment=train_env,
    process_count_per_node=process_count_per_node,
    compute_target=compute,
    node_count=node_count)

In [13]:
from azureml.pipeline.core import PipelineData

output_dir = PipelineData(name="training_output", 
                          datastore=dstore)

In [14]:
from azureml.contrib.pipeline.steps import ParallelRunStep

parallel_run_step = ParallelRunStep(
    name="many-models-training",
    parallel_run_config=parallel_run_config,
    inputs=[small_dataset_input],
    output=output_dir,
    arguments=['--target_column', 'Quantity', 
               '--n_test_periods', 6, 
               '--timestamp_column', 'WeekStarting', 
               '--stepwise_training', True])

In [16]:
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=[parallel_run_step])
run = experiment.submit(pipeline,tags=tags)
RunDetails(run).show()

Created step many-models-training [22726158][c4fd5941-c1b6-4c2a-9295-3c0c50daf59f], (This step will run and generate new outputs)
Created data reference oj_dataset_0 for StepId [e89ebf57][3ed92d44-8616-4b48-bb43-12f038ef80d1], (Consumers of this data will generate new runs.)
Submitted PipelineRun 45680a5e-7d08-4270-aec0-96993c60bdaf


WARNING - Converting non-string tag to string: (node_count: 5)
WARNING - Converting non-string tag to string: (process_count_per_node: 8)
WARNING - Converting non-string tag to string: (timeout: 500)


Link to Azure Machine Learning studio: https://ml.azure.com/experiments/github_training_pipeline/runs/45680a5e-7d08-4270-aec0-96993c60bdaf?wsid=/subscriptions/f97fb87f-32d7-4d7c-9bc5-ea43b4fea7ac/resourcegroups/ManyModelsRG/workspaces/ManyModelsAccelerator


NameError: name 'RunDetails' is not defined

In [ ]:
# run.wait_for_completion(show_output=True)

In [ ]:
published_pipeline = pipeline.publish(name='train_many_models',
                                     description='train many models',
                                     version='1',
                                     continue_on_step_failure=False)

In [ ]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
training_pipeline_id = published_pipeline.id

recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
recurring_schedule = Schedule.create(ws, name="training_pipeline_recurring_schedule", 
                            description="Schedule Training Pipeline to run on the first day of every month starting Jan 1, 2020 at 9AM",
                            pipeline_id=training_pipeline_id, 
                            experiment_name=experiment.name, 
                            recurrence=recurrence)